<a href="https://colab.research.google.com/github/consultant-decole/movie_stabilization_by_ffmpeg/blob/main/ffmpeg_mp4_stabilization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install python-dotenv to load environment variables from .env file
!pip install python-dotenv

In [ ]:
from dotenv import dotenv_values

# Load environment variables from .env file
env_vars = dotenv_values('.env')

# Access the variables
search_dir = env_vars.get('SEARCH_DIR')
output_dir = env_vars.get('OUTPUT_DIR')

print(f"search_dir loaded from .env: {search_dir}")
print(f"output_dir loaded from .env: {output_dir}")

In [ ]:
import glob

# search_dir is now loaded from .env in the previous cell

file_list_original = glob.glob(search_dir + '**/*.mp4', recursive = True) + \
                     glob.glob(search_dir + '**/*.MP4', recursive = True)

In [ ]:
import os
import subprocess
import json

for file in file_list_original:
  input_video_path = file
  # output_dir is now loaded from .env in a previous cell

  # --- Step 0: Detect original frame rate using ffprobe ---
  cmd_ffprobe = f'ffprobe -v error -select_streams v:0 -show_entries stream=avg_frame_rate -of json "{input_video_path}"'
  ffprobe_result = subprocess.run(cmd_ffprobe, shell=True, capture_output=True, text=True)

  detected_frate = None
  if ffprobe_result.returncode == 0:
      try:
          ffprobe_data = json.loads(ffprobe_result.stdout)
          if 'streams' in ffprobe_data and len(ffprobe_data['streams']) > 0:
              avg_frame_rate_str = ffprobe_data['streams'][0]['avg_frame_rate']
              # avg_frame_rate is typically in the format "num/den" (e.g., "30/1", "24000/1001")
              num, den = map(int, avg_frame_rate_str.split('/'))
              if den != 0:
                  detected_frate = num / den
                  print(f"Detected frame rate: {os.path.basename(input_video_path)}: {detected_frate} fps")
              else:
                  print(f"Error: Frame rate denominator is 0 for {os.path.basename(input_video_path)}.")
          else:
              print(f"Could not detect frame rate for {os.path.basename(input_video_path)}. ffprobe output: {ffprobe_result.stdout}")
      except json.JSONDecodeError as e:
          print(f"ffprobe JSON output parsing error: {os.path.basename(input_video_path)}: {e}")
      except Exception as e:
          print(f"An unexpected error occurred while getting frame rate: {os.path.basename(input_video_path)}: {e}")

  if detected_frate is None:
      print(f"Skipping processing for {os.path.basename(input_video_path)} due to unknown frame rate.")
      continue # Skip to the next file if frame rate cannot be detected

  # --- Existing Stabilization Logic (modified to use detected_frate) ---
  # Modified output filename to indicate black borders
  output_video_path_stabilized = output_dir + 'ffmpeg_stabilized_black_border_' + os.path.basename(file)

  if os.path.exists(output_video_path_stabilized):
      print(f"Skipping: {output_video_path_stabilized} already exists.")
  else:
      print(f"Starting stabilization process: {input_video_path} -> {output_video_path_stabilized}")
      # Modified params to include optzoom=0 for black borders

      # Parameter settings
      # optzoom=0 to completely disable zoom
      # interpol=bicubic for interpolation
      params = "smoothing=20:optzoom=0:interpol=bicubic"

      # params = "smoothing=20:optzoom=0:interpol=bicubic"
      transform_path = "transform.trf" # Temporary file

      # Filter description:
      # 1. Correct shake with vidstabtransform (no zoom)
      # 2. Expand canvas to 1.4x original size with pad, center (black background)
      #    width=iw*1.4 : 1.4 times original width / x=(ow-iw)/2 : centered
      filter_complex = f'vidstabtransform=input="{transform_path}":{params},pad=iw*1.4:ih*1.4:(ow-iw)/2:(oh-ih)/2:black'

      if not os.path.exists(input_video_path):
          print(f"Error: {input_video_path} not found. Please check the file name.")
      else:
          print("Step 1: Detecting shake...")
          cmd_detect = f'ffmpeg -y -i "{input_video_path}" -vf vidstabdetect=shakiness=5:accuracy=15:result="{transform_path}" -f null -'
          result_detect = subprocess.run(cmd_detect, shell=True, capture_output=True, text=True)
          if result_detect.returncode != 0:
              print(f"Error (vidstabdetect):\n{result_detect.stderr}")
          else:
              print(result_detect.stdout)

          # print("Step 2: Correcting and writing...")
          # # Use detected_frate for the stabilized output
          # cmd_transform = f'ffmpeg -y -i "{input_video_path}" -vf vidstabtransform=input="{transform_path}":{params} -r {detected_frate} -c:v libx264 -preset slow -crf 18 -c:a copy "{output_video_path_stabilized}"'
          print("Step 2: Correcting and writing with wider angle...")
          cmd_transform = f'ffmpeg -y -i "{input_video_path}" -vf "{filter_complex}" -r {detected_frate} -c:v libx264 -preset slow -crf 18 -c:a copy "{output_video_path_stabilized}"'
          result_transform = subprocess.run(cmd_transform, shell=True, capture_output=True, text=True)
          if result_transform.returncode != 0:
              print(f"Error (vidstabtransform):\n{result_transform.stderr}")
          else:
              print(result_transform.stdout)

          print("Done! Please check Google Drive.")
          if os.path.exists(transform_path):
              os.remove(transform_path)
